In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

from tqdm import trange, tqdm

sys.path.append('../..')
from svm import svm as SVM

import scipy
from statsmodels.stats.multitest import multipletests

#
import pandas as pd
import matplotlib.patches as mpatches
import scipy.stats as stats


Autosaving every 180 seconds


<ipython-input-1-2ad95b854a16>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# #######################################################
# #### RERUN SVM BUT LOWPASS FILTER LOCANMF COMPS #######
# #######################################################

#
def load_supersessions(animal_id, lockout):
    fname = os.path.join('/media/cat/4TBSSD/yuki/',
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    trials = np.vstack(data['trials'])
    random = np.vstack(data['random'])
    
    return trials, random

# 
# def filter_trace(trace, smooth_window=30):
#     box = np.ones(smooth_window)/smooth_window
#     trace_smooth = np.convolve(trace, box, mode='valid')
#     return trace_smooth

from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

#
def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

#
def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y

#
def butter_lowpass_lfilter(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter_lfilter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = lfilter(b, a, data)
    return y



#
def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)
        sig.append(res[1])

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    #print ("sig (BH corrected):", sig)
    edt = sig.shape[0]
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec

#
def plot_edt_distributions_box_plots(window,
                                     animal_id, 
                                     clrs_local, 
                                     lockout,
                                     filtered_flag):

    #
    codes = ['all', 'all-subsampled','lockedout']

    #
    vis = Visualize.Visualize()
    
    legend_size = 22
        
    # 
    shift= 15
    linestyles = ['-','-','-','-','-','-','-','-']

    #titles = ['all','lockout','concatenated']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    edts = []
    yvals = []

    # 
    ########## LOAD NON FILTERED FILE ##########
    # for some reason the manually recomputed EDT above gives some sprious results for IA2 (i.e. much too low)
    #   so just load the original computed edts from initial submission
    if False:
        fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_'+str(lockout)+'.npz')
        res1 = np.load(fname1, allow_pickle=True)['all_res_continuous']

        # remove artficat results below -15 sec
        # if False:
        #     idx = np.where(res1<-15)[0]
        #     res1 = np.delete(res1, idx)
        #     if idx.shape[0]>0:
        #         print ("errros found: ", idx.shape)
        
    else:
        # or load old data:
            
        fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_lockedout.npy')
        res1 = np.array(np.load(fname1,allow_pickle=True))
        times = res1[:,0]
        res2 = res1[:,1]
        idx = np.where(times==lockout)[0]
        res1 = np.hstack(res2[idx])
        #print (res1)
        
        # subselect every 4th sessoin to process only non-overlapping hybrid sessions
        if window==200:
            res1 = res1[::4]

            
    edts.append(np.hstack(res1))
    yvals.append(np.arange(len(res1)))
   # print (animal_id, "average EDT filtered: ", np.mean(edts))

    
        
    fname2 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_filtered_'+str(lockout)+'.npz')
    
    
    ########## LOAD 2ND FILE ##########
    res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']
            
    # 
    edts.append(np.hstack(res2))
    yvals.append(np.arange(len(res2)))

    #print ("edts: ", edts)
    print (animal_id, "average EDT filtered: ", np.mean(res2))
    
    # print differences between means
    #print (animal_id, "  diff in means: ", np.mean(res1) - np.mean(res2))
    
    # 
    my_dict = dict(Lockout = edts[0], 
                   Lockout_filter = edts[1],
                 )
    
    # 
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    ######################################################### 
    ################## SCATTER PLOTS ########################
    ######################################################### 
    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
        plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=10, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8

                )


    patches = []
    for c,code in enumerate(codes):
        patches.append(mpatches.Patch(color=clrs_local[c], label=code))

    ######################################################### 
    ###################### BOX PLOTS ########################
    ######################################################### 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)

    temp = np.hstack(yvals)
    ymax = np.max(temp)

    #
    plt.xlim(0.5, 4)

    #
    plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-5,-5],'--',linewidth=3,c='grey',alpha=.5)

    # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

    patches = []
    for p in [1]:
        res = stats.ks_2samp(edts[0], edts[p])
        #print ("res: ", res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color=clrs_local[p-1],label=label_))

    plt.xticks(rotation=45)
    #
    plt.legend(handles=patches,
               fontsize=15)
    plt.ylim(-15,0)


In [36]:
#######################################################
#### LOAD SUPER SESSIONS AND MEDIAN FILTER THEM #######
#######################################################
#
from scipy import ndimage, misc
def median_filter(data,
                  size=90):
    
    result = ndimage.median_filter(data, size=size)
    
    return result

lockouts = [15]
phase_shift = -int(30*(1/0.3))
print ("phase shift correction: ", phase_shift)
#
for lockout in lockouts:
    #lockout=15
    cutoff_freq = 0.3
    nyq_freq = 30

    #
    animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
    #animal_ids = ['IA3']

    # 
    for animal_id in animal_ids:
        fname_out = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_lfilter.npz')
        #
        #if os.path.exists(fname_out):
        #    continue
        
        print (animal_id, lockout)

        #
        trials, random = load_supersessions(animal_id, lockout)
        print (trials.shape)

        #
        for k in trange(trials.shape[0]):
            for p in range(trials.shape[2]):
                temp = butter_lowpass_filter_lfilter(trials[k,:,p],cutoff_freq, nyq_freq)
                trials[k,:,p] = np.roll(temp,phase_shift)
                temp = butter_lowpass_filter_lfilter(random[k,:,p],cutoff_freq, nyq_freq)
                random[k,:,p] = np.roll(temp,phase_shift)
                #
                #trials[k,:,p] = median_filter(trials[k,:,p])
                #random[k,:,p] = median_filter(random[k,:,p])
                
                

        np.savez(fname_out,
                 trials = trials,
                 random = random)
print ("DONE")

phase shift correction:  -100
IA1 15


  1%|          | 6/503 [00:00<00:08, 59.21it/s]

(503, 1800, 16)


100%|██████████| 503/503 [00:07<00:00, 64.12it/s]


IA2 15


...

AQ2 15


  1%|          | 7/594 [00:00<00:09, 63.67it/s]

(594, 1800, 16)


100%|██████████| 594/594 [00:09<00:00, 63.87it/s]


DONE


In [14]:
# def filter_sbs(data):
#     from scipy import signal
#     #data = random.random(2000)
#     b = signal.firwin(150, 0.004)
#     z = signal.lfilter_zi(b, 1) * data[0]
#     result = np.zeros(data.size)
#     for i, x in enumerate(data):
#         result[i], z = signal.lfilter(b, 1, [x], zi=z)
#     return result

# temp = filter_sbs(trials[10,:,11])
# print (temp.shape)

(1800,)


In [27]:
# ##############################################
# ##############################################
# ##############################################
# animal_id = "IA3"
# data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats.npz', allow_pickle=True)
# trials = np.vstack(data['trials'])
# print (trials.shape)

# #mfilter = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_medianfilter.npz', allow_pickle=True)
# #trials_mfilter = mfilter['trials']
# #print (trials_mfilter.shape)

# lfilter = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_lfilter.npz', allow_pickle=True)
# lfilter = lfilter['trials']
# print (lfilter.shape)

# filtfilt = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz.npz', allow_pickle=True)
# filtfilt = filtfilt['trials']
# print (filtfilt.shape)




(188, 1800, 16)
(188, 1800, 16)
(188, 1800, 16)


In [28]:
plt.figure()
plt.plot(trials[10,:,11], label='original')
#plt.plot(trials_mfilter[10,:,11], label='mfilter')
plt.plot(lfilter[10,:,11], label='lfilter (causal; phase corrected)')
plt.plot(filtfilt[10,:,11], label='filtfilt (noncausal)')
plt.legend()
plt.show()

In [3]:
#######################################################
#### LOAD SUPER SESSIONS AND LOWPASS FILTER THEM ######
#######################################################
#
lockouts = [0,6,9,12,15,18,21]

#
for lockout in lockouts:
    #lockout=15
    cutoff_freq = 0.3
    nyq_freq = 30

    #
    animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

    # 
    for animal_id in animal_ids:
        fname_out = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz.npz')
        #
        if os.path.exists(fname_out):
            continue
        
        print (animal_id, lockout)

        #
        trials, random = load_supersessions(animal_id, lockout)
        print (trials.shape)

        #
        for k in trange(trials.shape[0]):
            for p in range(trials.shape[2]):
                trials[k,:,p] = butter_lowpass_filter(trials[k,:,p],cutoff_freq, nyq_freq)
                random[k,:,p] = butter_lowpass_filter(random[k,:,p],cutoff_freq, nyq_freq)

        np.savez(fname_out,
                 trials = trials,
                 random = random)
print ("DONE")

DONE


In [37]:
#######################################################
########### RUN SVM ON FILTERED SUPER SESSIONS ########
#######################################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA3']

#
lockouts = [15]
#lockouts = [0,6,9,12,15,18,21]

for lockout in lockouts:
    for animal_id in animal_ids:
        # animal_id = 'AQ2'

        print ("animal id: ", animal_id)
        fname = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             #'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz.npz'
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_lfilter.npz'
                            
                            )

        # 
        data = np.load(fname, allow_pickle=True)
        trials = data['trials'].transpose(0,2,1)
        random = data['random'].transpose(0,2,1)
        print ("trials: ", trials.shape)

        # 
        svm = SVM.PredictSVMChoiceSuperSession()
        svm.xvalidation = 10
        svm.parallel = True

        # window parameters
        svm.window = 30              # seconds of prediction window, backwards in time in seconds; e.g. -30sec ... +30sec
        svm.lockout_window = 10      # time for locking out other pulls 
        svm.sliding_window = 30      # number of frames in sliding window

        # pca params
        svm.pca_flag = True
        svm.pca_var = 0.95        # this is default for now for nComp = 20

        # svm parameters
        svm.xvalidation = 10      # KFold xvalidation step
        svm.data_split = 0.8      # split 80/20 data for training
        svm.method = 'sigmoid'    # method used for computing SVM

        # run-time parameters
        svm.parallel = True
        svm.n_cores = svm.xvalidation
        svm.min_trials = 30
        svm.overwrite = False
        svm.gpu_flag = False

        #
        fname_out = fname

        # this function is the latest version for filtered data
        # it saves the output to fname_save = fname_out.replace('.npz','_'+str(k)+"_super_res.npz")
        svm.predict3(trials, random, fname_out)

print ("DONE")

animal id:  IA1
trials:  (503, 16, 1800)
   start:  0  end:  200  /  503
   start:  50  end:  250  /  503
   start:  100  end:  300  /  503
   start:  150  end:  350  /  503
   start:  200  end:  400  /  503
   start:  250  end:  450  /  503
   start:  300  end:  500  /  503
   start:  350  end:  550  /  503
   start:  400  end:  600  /  503
   start:  450  end:  650  /  503
animal id:  IA2
trials:  (564, 16, 1800)
   start:  0  end:  200  /  564
   start:  50  end:  250  /  564
   start:  100  end:  300  /  564
   start:  150  end:  350  /  564
   start:  200  end:  400  /  564
   start:  250  end:  450  /  564
   start:  300  end:  500  /  564
   start:  350  end:  550  /  564
   start:  400  end:  600  /  564
   start:  450  end:  650  /  564
   start:  500  end:  700  /  564
animal id:  IA3
trials:  (188, 16, 1800)
animal id:  IJ1
trials:  (261, 16, 1800)
   start:  0  end:  200  /  261
   start:  50  end:  250  /  261
   start:  100  end:  300  /  261
   start:  150  end:  350  / 

In [51]:
###############################################
###### MAKE FIRST DECODING TIMES FILES ########
###############################################
import scipy
from statsmodels.stats.multitest import multipletests

root_dir = '/media/cat/4TBSSD/yuki'

#
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA3']
window = 50

#
#lockouts = [0,6,9,12,15,18,21]
#lockouts = [3,6,9,12,15,18,21]
lockouts = [15]
for lockout in lockouts:
    for animal_id in animal_ids:

        # load
        ctr=0
        edts = []
        for n in trange(0, 10000, window):
            #print (n)
            #fname_out = '/media/cat/4TBSSD/yuki/'+animal_id+"_edts_locked_out_filtered_"+str(lockout)+"_lfilter_phaseshifted.npz"
            fname_out = '/media/cat/4TBSSD/yuki/'+animal_id+"_edts_locked_out_"+str(lockout)+"_rerun.npz"
            
            #if os.path.exists(fname_out):
            #    continue
            
            fname = os.path.join(root_dir,
                                 animal_id,
                                 'super_sessions',
                                 'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_'+str(n)+'_super_res.npz'
                                # 'alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_super_res_subsample_'
                                 #'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz_'+str(n)+'_super_res.npz'
                                 #'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_lfilter_'+str(n)+'_super_res.npz'
                                
                                )

            try:
                data = np.load(fname,
                               allow_pickle=True)
            except:
                print ("missing: ", fname)
                print ("Done ...  ",animal_id)
                break

            # load accuracy
            acc = data['accuracy']

            # 
            for k in range(acc.shape[1]):
                acc[:,k] = butter_lowpass_filter(acc[:,k],0.5, 30)

            # 
            edt = compute_significance(acc[:900])

            # must add 1 sec because of SVM works in 1 sec chunks
            edt = edt+1
            
            # remove any bad decoding values
            if edt>0:
                edt=0

            #
            edts.append(edt)
            ctr+=1
            
        #
        print (animal_id, lockout, " total # of supersessions: ", ctr) #, ', edts', edts)
        #print ("Fname out: ", fname_out)

        #
        np.savez(fname_out, 
                 all_res_continuous = edts)

        #
        print ("")
        print ("")

    
#
print ("DONE ALL") 

  0%|          | 0/200 [00:00<?, ?it/s]

missing:  /media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_500_super_res.npz
Done ...   IA1
IA1 15  total # of supersessions:  10




  0%|          | 0/200 [00:00<?, ?it/s]

missing:  /media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_550_super_res.npz
Done ...   IA2
IA2 15  total # of supersessions:  11




  0%|          | 0/200 [00:00<?, ?it/s]

...

missing:  /media/cat/4TBSSD/yuki/IJ1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_250_super_res.npz
Done ...   IJ1
IJ1 15  total # of supersessions:  5




  0%|          | 0/200 [00:00<?, ?it/s]

missing:  /media/cat/4TBSSD/yuki/IJ2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_150_super_res.npz
Done ...   IJ2
IJ2 15  total # of supersessions:  3




  6%|▌         | 12/200 [00:03<01:01,  3.04it/s]

missing:  /media/cat/4TBSSD/yuki/AQ2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_600_super_res.npz
Done ...   AQ2
AQ2 15  total # of supersessions:  12


DONE ALL


In [ ]:
fname2:  /media/cat/4TBSSD/yuki/IJ1_edts_locked_out_filtered_0.npz

In [57]:
###############################################################
###############################################################
###############################################################

def plot_edt_distributions_box_plots_filtered(window,
                                              animal_id, 
                                              clrs_local, 
                                              lockouts,
                                              filtered_flag):

    #
    codes = ['all', 'all-subsampled','lockedout']

    #
    vis = Visualize.Visualize()
    
    legend_size = 22
        
    # 
    shift= 15
    linestyles = ['-','-','-','-','-','-','-','-']

    #titles = ['all','lockout','concatenated']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    edts = []
    yvals = []

    # 
    ########## LOAD NON FILTERED FILE ##########
    # for some reason the manually recomputed EDT above gives some sprious results for IA2 (i.e. much too low)
    #   so just load the original computed edts from initial submission
    
    # 
    for lockout in lockouts:
         
#         fname2 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_'+str(lockout)+'_rerun.npz')
        
        
#         #print ("fname2: ", fname2)

#         ########## LOAD 2ND FILE ##########
#         res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']

#         # 


        fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_lockedout.npy')
        res1 = np.array(np.load(fname1,allow_pickle=True))
        times = res1[:,0]
        res2 = res1[:,1]
        idx = np.where(times==lockout)[0]
        res1 = np.hstack(res2[idx])+1
        edts.append(np.hstack(res1))
        yvals.append(np.arange(len(res1)))        
        
        ###########################
        # causal filter filtfilt
        fname2 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_filtered_'+str(lockout)+'.npz')
        
        
        #print ("fname2: ", fname2)

        ########## LOAD 2ND FILE ##########
        res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']

        # 
        edts.append(np.hstack(res2))
        yvals.append(np.arange(len(res2)))

        fname2 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_filtered_'+str(lockout)+'_lfilter_phaseshifted.npz')
         
        ########## LOAD 2ND FILE ##########
        # noncausla filter lfilter + phase shift
        res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']

        # 
        edts.append(np.hstack(res2))
        yvals.append(np.arange(len(res2)))
            
        #print ("edts: ", edts)
        #print (animal_id, "average EDT filtered: ", np.mean(res2))


    # 
    my_dict = dict(
                   three = edts[0], 
                   six = edts[1], 
                   nine = edts[2], 
                   #twelve = edts[3], 
                   #fifteen = edts[4], 
                   #eighteen = edts[5], 
                   #three = edts[0], 
                   #Lockout_filter = edts[1],
                 )
    
    # 
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()
    means =np.mean(data,0)
    
    print ("Means: ", means)
    
    #print ("15 sec lockout: ", edts[4])

    ######################################################### 
    ################## SCATTER PLOTS ########################
    ######################################################### 
    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
        plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=10, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8

                )


    patches = []
    for c,code in enumerate(codes):
        patches.append(mpatches.Patch(color=clrs_local[c], label=code))

    ######################################################### 
    ###################### BOX PLOTS ########################
    ######################################################### 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                 showmeans = True,
                 flierprops=flierprops)

    temp = np.hstack(yvals)
    ymax = np.max(temp)

    #
    plt.xlim(0.5, 5.5)

    #
    plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-5,-5],'--',linewidth=3,c='grey',alpha=.5)

    #
    plt.xticks(rotation=45)

    # 
    plt.ylim(-15,0)
    
    return means
#
window = 50
#############################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA3']
clrs_local = ['darkblue','purple','darkred','orange','yellow']
filtered_flag = True

#
ctr=0
fig=plt.figure(figsize=(20,5))
means = []
#lockouts = [0,6,9,12,15]
lockouts = [15]
for animal_id in animal_ids:
    ax=plt.subplot(1,6,ctr+1)
    
    mean = plot_edt_distributions_box_plots_filtered(window, 
                                                    animal_id,
                                                    clrs_local,
                                                    lockouts,
                                                    filtered_flag)
   # print (animal_id, mean)
    means.append(mean)
    
    ctr+=1
    
plt.suptitle("Lockout "+str(lockout))
#######################################
if False:
    plt.savefig('/home/cat/filtered_EDTs_nonoverlapping.svg')
    plt.close()
else:
    plt.show()

Means:  three   -0.926667
six     -3.500000
nine    -2.283333
dtype: float64
Means:  three   -2.139394
six     -4.854545
nine    -3.984848
dtype: float64
Means:  three   -1.266667
six     -6.950000
nine    -5.525000
dtype: float64
Means:  three   -0.873333
six     -5.006667
nine    -3.226667
dtype: float64
Means:  three   -0.644444
six     -3.000000
nine    -3.288889
dtype: float64
Means:  three   -1.491667
six     -3.700000
nine    -3.005556
dtype: float64


In [9]:
Means:  three   -2.283333
dtype: float64
Means:  three   -3.984848
dtype: float64
Means:  three   -5.525
dtype: float64
Means:  three   -3.226667
dtype: float64
Means:  three   -3.288889
dtype: float64
Means:  three   -3.005556
dtype: float64





-3.499999999


In [55]:
##################################
############## FIG 6J ############
##################################

# generate fig 6h revision manuscript

fig=plt.figure()
t = np.array([3,6,9,12,15])
for k in range(6):
    mm = means[k]#.to_numpy()
    #print (mm)
    plt.plot(t, mm, 
             label=animal_ids[k],
             linewidth=5)
    plt.scatter(t, mm,
               s=100)
    
    
plt.plot([3,15],[0,0],'--',c='grey')
plt.plot([3,15],[-3,-3],'--',c='grey')
plt.plot([3,15],[-5,-5],'--',c='grey')
plt.plot([3,15],[-10,-10],'--',c='grey')

plt.ylim(-11,0)
#plt.legend()
if True:
    plt.savefig('/home/cat/mean_fits.svg')
    plt.close()
else:
    plt.show()
    
    

In [18]:
# ############################################
# ############################################
# ############################################
fname = '/home/cat/IA1_edts_lockedout.npy'
data = np.load(fname, allow_pickle=True)
lockouts = data[:,0]
print (lockouts)


[0 6 9 12 15 18 21 24 27 30]
[0
 list([-2.8666666666666667, -3.0, -3.466666666666667, -3.1, -7.566666666666666, -7.533333333333333, -3.533333333333333, -3.533333333333333, -3.2333333333333334, -3.3333333333333335, -3.066666666666667, -2.933333333333333, -3.4, -2.8, -3.0, -2.7333333333333334, -2.566666666666667, -2.3, -1.9, -1.5666666666666667, -1.6666666666666667, -1.7666666666666666, -1.3666666666666667, -1.4, -1.3, -0.7666666666666667, -1.4])]


In [156]:
# #############################################
# #############################################
# #############################################
# session_name = 'IA1pm_Feb23_30Hz'

# trials = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_trial_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
# trials2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name+'_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
# random = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
# random2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')

# print (trials.shape)
# print (random.shape)

# #
# t = np.arange(trials.shape[2])/30.-30

# #
# plt.figure(figsize=(20,10))
# ax1=plt.subplot(2,1,1)
# plt.plot(t,trials2[0,0].T,c='blue',alpha=1,label='unfiltered')
# plt.plot(t,trials[0,0].T,c='green',alpha=1,label='filtered')
# plt.legend(fontsize=20)
# plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
# plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
# plt.xlabel("Time (sec)")
# plt.ylabel("Trial")
# plt.xlim(-30,30)
# plt.ylim(-10,10)    
# #
# ax1=plt.subplot(2,1,2)
# plt.plot(t,random2[0,0].T,c='red',alpha=1,label='unfiltered')
# plt.plot(t,random[0,0].T,c='green',alpha=1,label='filtered')
# plt.legend(fontsize=20)
# plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
# plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
# plt.xlabel("Time (sec)")
# plt.ylabel("Random")

# if False:
#     plt.savefig('/home/cat/trials.svg')
#     plt.close()
# plt.show()


(81, 8, 1801)
(76, 8, 1801)


In [51]:
############################################
########## QUICK AND DIRTY CHECKS ##########
############################################
#session_name = 'IA1pm_Feb23_30Hz'
fnames = [

    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_400_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_350_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_300_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_250_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_150_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_100_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_50_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz'

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
    '/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz',
    '/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
]

# 
fig=plt.figure()
ax=plt.subplot(111)
for ctr,fname in enumerate(fnames):
    
    data = np.load(fname)
    
    
    predictions = data['predictions']
    print ("n_trials: ", predictions.shape[1])
    acc = data['accuracy']
    #length = 1801-acc.shape[0]
    #acc = np.vstack(
    #                 (np.zeros((length,10), dtype=np.float32), 
    #                  acc))
    print (acc.shape)
    t = np.arange(acc.shape[0])/30-29
    mean = acc.mean(1)
    std = np.std(acc,1)
    plt.plot(t,mean, label=fname)
    ax.fill_between(t, mean+std,
                    mean-std,
                    alpha=.2)

plt.plot([-30,30],[0.5,0.5],'--',c='grey')
plt.ylim(0,1)
plt.xlim(-20,0)
#plt.legend(fontsize=6)
if False:
    plt.savefig('/home/cat/filtered_IA3.svg')
    plt.close()
else:
    plt.show()


####
# plt.figure()
# d1 = np.load(fname)
# acc1 = d1['accuracy']
# print (acc1.shape)

# plt.plot(acc1.mean(1), label='all freq')

# d2 = np.load(fname_filtered)
# acc2 = d2['accuracy']
# print (acc2.shape)

# plt.plot(acc2.mean(1), label='<0.3hz')
# plt.legend()
# plt.show()

n_trials:  400
(900, 10)
n_trials:  400
(1770, 10)


In [42]:
fname = '/media/cat/4TBSSD/yuki/IA1/first_decoding_time_0.3hzFilter.npz'
data=np.load(fname)

sessions = data['all_session_names']
print (sessions.shape)

(53,)
